In [1]:
import os
import time
import sys
import json
import numpy as np
import ipyvolume as ipv
from pathlib import Path
import numpy as np
import random
from datetime import datetime

from tyssue import Sheet, config
from tyssue.io import hdf5
from tyssue.generation import ellipsoid_sheet
from tyssue.draw.ipv_draw import view_ipv
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.dynamics import effectors, units
from tyssue.dynamics.sheet_gradients import height_grad
from tyssue.dynamics.factory import model_factory
from tyssue.solvers.sheet_vertex_solver import Solver
from tyssue.behaviors.events import EventManager
from tyssue.topology.sheet_topology import remove_face
from tyssue.utils.decorators import do_undo
from tyssue.utils import to_nd
from tyssue.io import hdf5


from invagination.ellipsoid import EllipsoidBGeometry as geom
from invagination.ellipsoid import VitellineElasticity
from invagination.delamination import (define_ovoid_mesoderm,
                                       initiate_monitoring_process,
                                       check_enter_in_process,
                                       check_tri_faces,
                                       type1_transition)

from invagination.basic_plot import mesoderm_position


import matplotlib.pyplot as plt

%matplotlib inline

SIM_DIR = Path('/home/admin-suz/Documents/Simulations')


In [2]:
import datetime

today = datetime.date.today()

sim_save_dir = SIM_DIR/f'{today.isoformat()}_principal_results'

try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [ ]:
#%pdb

In [3]:
class RadialTension(effectors.AbstractEffector):
    
    dimensions = units.line_tension
    magnitude = 'radial_tension'
    label = 'Apical basal tension'
    element = 'vert'
    specs = {'vert':{'is_active',
             'height',
             'radial_tension'}}

    @staticmethod
    def energy(eptm):
        return eptm.vert_df.eval(
            'height * radial_tension * is_active')
         
    @staticmethod
    def gradient(eptm):
        grad = height_grad(eptm) * to_nd(
            eptm.vert_df.eval('radial_tension'), 3)
        grad.columns = ['g'+c for c in eptm.coords]
        return grad, None
    
    
EllipsoidSModel = model_factory(
    [
    RadialTension,
    VitellineElasticity,
    effectors.LineTension,
    effectors.FaceContractility,
    effectors.FaceVolumeElasticity,
    ], effectors.FaceAreaElasticity)


EllipsoidBModel = model_factory(
    [
    RadialTension,
    VitellineElasticity,
    #effectors.LineTension,
    effectors.FaceContractility,
    effectors.FaceAreaElasticity,
    effectors.CellVolumeElasticity,
    ], effectors.FaceAreaElasticity)


model = EllipsoidBModel


In [4]:
dsets = hdf5.load_datasets('../data/hf5/ellipsoid_sheet_init.hf5',
                           data_names=['vert', 'edge', 'face', 'cell'])


with open('../data/json/ellipsoid.json', 'r+') as fp:
    specs = json.load(fp)

sheet = Sheet('ellipse', dsets, specs)

# Modify some initial value
sheet.settings['threshold_length'] = 1e-3
sheet.settings['vitelline_space'] = 0.2
sheet.vert_df['radial_tension'] = 0.
sheet.cell_df['prefered_vol'] = 4539601.384437251
sheet.cell_df['vol_elasticity'] = 3.e-6

fig, ax = quick_edge_draw(sheet, coords=list('zx'))

In [5]:
solver_kw = {'minimize': {'method': 'L-BFGS-B',
                          'options': {'ftol': 1e-8,
                                      'gtol': 1e-8}}}

## Define cells in the mesoderm

In [6]:
# Define rectangular mesoderm
#define_cell_in_mesoderm(sheet, 0.3, 0.7 ,0.05, 0.95)

# Define ovoid mesoderm
define_ovoid_mesoderm(sheet, 0, 0, 145, 40)

mesoderm = sheet.face_df[sheet.face_df.is_mesoderm].index
delaminating_cells = sheet.face_df[sheet.face_df['is_mesoderm']].index



print('number of apoptotic cells: {}'.format(delaminating_cells.size))
mesoderm_position(sheet, delaminating_cells)

## Processus functions

In [7]:
import logging
logger = logging.Logger('event_log')


@do_undo
def run_sim(sheet, mesoderm, geom, model, dirname, largeur_gauss, densite_proba):
    solver = Solver
    
    # logger initiation
    event_logfile = os.path.join(dirname, 'events.log')
    hdlr = logging.FileHandler(event_logfile)
    hdlr.setLevel('INFO')
    logger.addHandler(hdlr)
        
    #Initiate manager
    manager = EventManager('face')

    
    t=0
    stop = 200
    while manager.current and t < stop:
        # Clean radial tension on all vertices
        sheet.vert_df['radial_tension'] = 0
        manager.execute(sheet)
        res = solver.find_energy_min(sheet, geom, model, **solver_kw)
        
        # add noise on vertex position to avoid local minimal.
        sheet.vert_df[['x', 'y']] += np.random.normal(scale=1e-3, size=(sheet.Nv, 2))
        geom.update_all(sheet)

        figname = os.path.join(
            dirname, 'invagination_{:04d}.png'.format(t))
        hdfname = figname[:-3]+'hf5'
        hdf5.save_datasets(hdfname, sheet)      
        
        
        # Add cells in delamination process if they are authorized
        check_enter_in_process(sheet, manager, mesoderm, 
                                    t, largeur_gauss, densite_proba)   
        
        
        # Add cells with initially 3 neighbourghs to be eliminated
        check_tri_faces(sheet, manager)
        # Add T1 transition for face with at least one edge shorter than critical length
        [manager.append(type1_transition, f, kwargs=sheet.settings['T1']) for f in sheet.edge_df[
            sheet.edge_df['length'] < sheet.settings['T1']['critical_length']]['face'].unique()]
    
    
        manager.update()
        t+=1
        
        
    logger.removeHandler(hdlr)
    return sheet


In [8]:
def delamination_process(sheet, contractility_rate, critical_area, 
                         radial_tension, largeur_gauss, densite_proba, nb_iteraction_max):

    # Directory definition 
    dirname = '{}_contractility_{}_critical_area_{}_radialtension'.format(
                contractility_rate, critical_area, radial_tension)
    dirname = os.path.join(sim_save_dir, dirname)
    
    print('starting {}'.format(dirname))
    try:
        os.mkdir(dirname)
    except IOError:
        pass
    
    settings = {'contract_rate': contractility_rate,
            'critical_area': critical_area,
            'radial_tension': radial_tension,
            'nb_iteration':0,
            'nb_iteration_max':nb_iteraction_max,
            'contract_neighbors':True,
            'critical_area_neighbors':10,
            'geom': geom}

    solver = Solver
    
    # Add some information to the sheet
    sheet2 = sheet.copy(deep_copy=True)
    
    sheet2.face_df['id'] = sheet2.face_df.index.values
    sheet2.settings['delamination'] = settings
    
    settings2 = {'critical_length':0.3}
    sheet2.settings['T1'] = settings2
    #""" Initiale find minimal energy
    # To be sure we are at the equilibrium
    res = solver.find_energy_min(sheet2, geom, model, **solver_kw)
   
    sheet2 = run_sim(sheet2, delaminating_cells, geom, model, dirname, largeur_gauss, densite_proba)

    print('{} done'.format(dirname))
    print('~~~~~~~~~~~~~~~~~~~~~\n')
    

## Sequential execution 

In [9]:
from datetime import datetime

global_start=datetime.now()
print ("start : " + str(global_start))

length_gauss = 13
n = 2
nb_iter = 30
critical_area = 5

radial_tension = [0, 25, 50, 75, 100]
contractility_percent = [4, 8, 16, 32, 64]
contractility_rate = [1+c/100 for c in contractility_percent]

for rad in radial_tension:
    for contracts in contractility_rate:
        print ('rad : ' + str(rad)+ '\tcontract : ' + str(contracts))
        delamination_process(sheet, contracts, critical_area, rad, length_gauss,n , nb_iter)

global_end = datetime.now()
print ("end : " + str(global_end))
print ('Duree totale d execution : \n\t\t')
print (global_end-global_start)

## Parallel execution

In [ ]:

from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

global_start=datetime.now()
print ("start : " + str(global_start))
num_cores = multiprocessing.cpu_count()

length_gauss = 17
n = 2
nb_iter = 30
critical_area = 5

radial_tension = [100]
contractility_percent = [ 32, 64, 8, 16,]
contractility_rate = [1+c/100 for c in contractility_percent]


contractility_rate, radial_tension = np.meshgrid(contractility_rate, radial_tension)

results = Parallel(n_jobs=2)(delayed(delamination_process)(
    sheet, c, critical_area, r, length_gauss,n , nb_iter) for c, r in zip(contractility_rate.ravel(), radial_tension.ravel()))

global_end = datetime.now()
print ("end : " + str(global_end))
print ('Duree totale d execution : \n\t\t')
print (global_end-global_start)
